<a href="https://colab.research.google.com/github/jacobashwall/Battlegrounds/blob/main/tweets_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model

##Init

In [ ]:
import pandas as pd
import random
import re
import h5py
import math
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from tabulate import tabulate
from google.colab import files
import os
import time
from google.colab import drive
import psutil
import fcntl
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN
from multiprocessing import Process
import tensorflow as tf



In [ ]:
!sudo add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!sudo apt-get update -qq 2>&1 > /dev/null
!sudo apt -y install -qq google-drive-ocamlfuse 2>&1 > /dev/null
!google-drive-ocamlfuse



debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
/usr/bin/xdg-open: 869: www-browser: not found
/usr/bin/xdg-open: 869: links2: not found
/usr/bin/xdg-open: 869: elinks: not found
/usr/bin/xdg-open: 869: links: not found
/usr/bin/xdg-open: 869: lynx: not found
/usr/bin/xdg-open: 869: w3m: not found
xdg-open: no method available for opening 'https://accounts.google.com/o/oauth2/auth?client_id=564921029129.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fgd-ocaml-auth.appspot.com%2Foauth2callback&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&

In [ ]:
!sudo apt-get install -qq w3m # to act as web browser 
!xdg-settings set default-web-browser w3m.desktop # to set default browser
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package w3m.
(Reading database ... 128280 files and directories currently installed.)
Preparing to unpack .../w3m_0.5.3-37ubuntu0.1_amd64.deb ...
Unpacking w3m (0.5.3-37ubuntu0.1) ...
Setting up w3m (0.5.3-37ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for mime-support (3.64ubuntu1) ...
/content
/content/drive
/content
/
Access token retrieved correctly.


##Vectors

###Split

In [ ]:
TRAIN_PERCENTAGE = 75  # the percentage to divide the dataset to training and testing datasets
READ_COLUMNS=[ #the columns that are read from the csv file
"מס סידורי"
,"שם משתמש"
,"גיל"
,"קבוצת גיל"
,"מין"
,"0-ימין"
,"1-ימין מרכז"
,"2-שמאל מרכז"
,"3-שמאל"
,"0-ליכוד"
,"1-יש עתיד"
,"2-הציונות הדתית"
,"3-מרצ"
,"4-תקווה חדשה"
,"5-ימינה"
,"6- ישראל ביתנו"
,"7-כחול לבן"
,"8-ש\"ס"
,"9-הרשימה המשותפת"
,"10-העבודה"
,"11-רע\"מ"
,"12-ח\"כ בודד"
,"0-ספרדי"
,"1-אשכנזי"
,"דתי אורתודוקסי"
,"לא חרדי-0"
,"חרדי-1"
,"יהודי"
]

In [ ]:
def read_tweets():
  """
  reads the data from the tweets sheet
  :return: the data as pandas data set
  """
  return pd.read_csv('/content/drive/MyDrive/model_files/data/TweetsRes - The 24 Knesset_ Tweets.csv', encoding='utf-8-sig')


def read_knesset_members():
  """
  reads the data from the knesset members sheet
  :return: the data as pandas data set
  """
  return pd.read_csv('/content/drive/MyDrive/model_files/data/TweetsRes - The 24 Knesset_ information.csv', usecols= READ_COLUMNS, encoding='UTF8')

def write_train_tweets(train_tweets_set):
  """
  writes the tweets of the training tweets dataset
  """
  train_tweets_set.to_csv('/content/drive/MyDrive/model_files/data/train_tweets_set.csv', encoding='utf-8-sig',index=False)
  train_tweets_set.to_hdf('/content/drive/MyDrive/model_files/data/train_tweets_set.h5', key='train_tweets_set', mode='w')


def write_train_knesset_members(train_knesset_members_set):
  """
   writes the data of the training knesset members dataset
  """
  train_knesset_members_set.to_csv('/content/drive/MyDrive/model_files/data/train_knesset_members_set.csv', encoding='utf-8-sig',index=False)
  train_knesset_members_set.to_hdf('/content/drive/MyDrive/model_files/data/train_knesset_members_set.h5', key='train_knesset_members_set', mode='w')

def write_test_tweets(test_tweets_set):
  """
   writes the tweets of the testing tweets dataset
  """
  test_tweets_set.to_csv('/content/drive/MyDrive/model_files/data/test_tweets_set.csv', encoding='utf-8-sig',index=False)
  test_tweets_set.to_hdf('/content/drive/MyDrive/model_files/data/test_tweets_set.h5', key='test_tweets_set', mode='w')

def write_test_knesset_members(test_knesset_members_set):
  """
  writes the data of the testing knesset members dataset
  """
  test_knesset_members_set.to_csv('/content/drive/MyDrive/model_files/data/test_knesset_members_set.csv', encoding='utf-8-sig',index=False)
  test_knesset_members_set.to_hdf('/content/drive/MyDrive/model_files/data/test_knesset_members_set_set.h5', key='test_knesset_members_set', mode='w')

def split(knesset_member_ds, tweets_ds, train_percentage):
  """
  splits by a given percentage the knesset members and then splits the tweets according to their author into train
  and test tweets.
  :param tweets_ds: the tweets' dataset
  :param knesset_member_ds: the knesset members dataset
  :param train_percentage: the parentage of the split
  :return the split knesset members in their corresponding tweets
  """
  random.seed(42)  # in order to get consistent data
  number_of_knesset_members = knesset_member_ds['מס סידורי'].count()
  number_of_train = int((number_of_knesset_members * (train_percentage / 100)))
  # get the serial number of the training and test set knesset members randomly
  train_serial = random.sample(range(1, number_of_knesset_members + 1), number_of_train)
  test_serial = [serial for serial in range(1, number_of_knesset_members + 1) if serial not in train_serial]
  # get the test and train the knesset members information by the serial number
  train_knesset_members_set = knesset_member_ds.loc[knesset_member_ds['מס סידורי'].isin(train_serial)]
  test_knesset_members_set = knesset_member_ds.loc[knesset_member_ds['מס סידורי'].isin(test_serial)]
  # get the test and train knesset members username
  train_knesset_members_set_usernames = train_knesset_members_set['שם משתמש']
  test_knesset_members_set_usernames = test_knesset_members_set['שם משתמש']
  # get the test and train tweets by the knesset members username
  train_tweets_set = tweets_ds.loc[tweets_ds['User'].isin(train_knesset_members_set_usernames)]
  test_tweets_set = tweets_ds.loc[tweets_ds['User'].isin(test_knesset_members_set_usernames)]
  return train_knesset_members_set, train_tweets_set, test_knesset_members_set, test_tweets_set

In [ ]:
tweets_ds = read_tweets()
knesset_member_ds = read_knesset_members()
train_knesset_members_set, train_tweets_set, test_knesset_members_set, test_tweets_set = split(knesset_member_ds,
                                                                                                  tweets_ds,
                                                                                                  TRAIN_PERCENTAGE)
write_train_tweets(train_tweets_set)
write_train_knesset_members(train_knesset_members_set)
write_test_tweets(test_tweets_set)
write_test_knesset_members(test_knesset_members_set)

/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['User', 'Tweet'], dtype='object')]

  pytables.to_hdf(


###TF-IDF

In [ ]:
NT = 3  # the minimum nuber of tweets a word in tf idf should show
SPECIAL_CHARCATERS_REMOVAL=r"(?u)\b\w+[\"\'״]*\w\b" # tokenize by " " and "/n" and allow abbreviations
TOKEN_PATTERN="[^ \n]+" # baseline tokenize, only by space and end of row

# Gets all the stop words from the file
pattern = re.compile(SPECIAL_CHARCATERS_REMOVAL)
with open("/content/drive/MyDrive/model_files/data/stop_words.txt", 'r') as f:
        STOP_WORDS = pattern.findall(f.read())

# Insert all the abbreviations into a dictionary
ABBREVIATIONS = {}
with open("/content/drive/MyDrive/model_files/data/abbreviations.txt") as f:
    for line in f:
        key, value = line.split("-")
        ABBREVIATIONS[key.strip()] = value.strip()

# To preprocess open abbrevaitons
def expand_abbreviations(text):
    """
    a function that gets a text and replace all abbreviations with thier expnaded form
    """
    for match in re.finditer(SPECIAL_CHARCATERS_REMOVAL, text):
        abbreviation = match.group()
        normelized_abbreviation=abbreviation
        # normalize all " 
        if "״" in abbreviation:
          normelized_abbreviation=abbreviation.replace("״","\"")
        if "''" in abbreviation:
          normelized_abbreviation=abbreviation.replace("''","\"")
        # if its in the, replace with expanded form
        if normelized_abbreviation in ABBREVIATIONS:
            text = text.replace(abbreviation, ABBREVIATIONS[normelized_abbreviation])
    return text

VECTORIZERS={ # the tf-idf vectorizers by preprocessing method
    'N' : TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN),#None, baseline
    # only one preprocessing method
    'S':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS),#Stop word removal
    'C':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL),#special Characters removal
    'O':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN, preprocessor=expand_abbreviations),#Open abbreviations
    # pairing preprocssing methods
    #'SC':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS),
    #'SO':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
    #'CO':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,preprocessor=expand_abbreviations),
    # all preprocessing methods
    #'SCO':TfidfVectorizer(min_df=NT,lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
}

In [ ]:
def read_train_tweets():
  """
  reads the data from the train tweets sheet
  :return: the data as pandas data set
  """
  
  store = pd.HDFStore('/content/drive/MyDrive/model_files/data/train_tweets_set.h5')
  df = store.select('train_tweets_set')
  store.close()

  columns_to_include = ["Tweet"]
  df = df[columns_to_include]
  df.dropna(subset=["Tweet"], inplace=True)
  return df

def get_words(tweets,preprocessing):
  """
  returns all the tweets words by the preprocessing method
  """
  corpus=tweets
  if "O" in preprocessing:
    corpus=corpus.apply(expand_abbreviations)
  
  if "C" in preprocessing:
    pattern = re.compile(SPECIAL_CHARCATERS_REMOVAL)
  else:
    pattern = re.compile(TOKEN_PATTERN)

  corpus_words=[]
  for document in tqdm(corpus):
      corpus_words+= pattern.findall(document)
  return corpus_words

def get_terms():
  columns= ["nt", "f", "tf","idf","tfidf"]
  meanings= ["Number of different documents that the word appears in.",
   "Number of appearances of the word in all documents.",
   "Term frequency.",
   "Inverse document frequency.",
   "Term fruquency multiplied by inverse document frequency."]
  return columns, meanings
   
  
def write_tf_idf_chart(preprocessing, vectorizer):
  """
  writes the tf idf chart by the given preprocessing vectorizer
  """
  train_tweets = read_train_tweets()
  sparse_matrix = vectorizer.fit_transform(train_tweets["Tweet"])
  dense_matrix = sparse_matrix.toarray()

  # Get the vocabulary from the vectorizer object
  vocab = vectorizer.get_feature_names_out()
  df = pd.DataFrame(columns=['word', 'nt', 'f', 'tf', 'idf', 'tf-idf',' ','Column','Meaning'])

  # Gather all the tweets words to calculate words frequency
  print("Getting words from tweets:")
  tweets_words=get_words(train_tweets["Tweet"],preprocessing)
  print("done!")

  print("calculating 'nt' and 'f':")
  for word in tqdm(vocab):
      # The column of tf-idf values of the specific word, as the rows are the tweets 
      word_column=dense_matrix[:, vectorizer.vocabulary_[word]]
      # Count the number of tweets the word appears in
      nt = np.count_nonzero(word_column)
      # Count the number of times the word is used in all tweets
      f = tweets_words.count(word)
      # append all the data we collected so far to each word to the data frame
      df = df.append({'word': word, 'nt': nt, 'f': f, 'tf': 0, 'idf': 0, 'tf-idf': 0}, ignore_index=True)
  print("done!")

  # the numebr of words in total is the sum of the f of each word
  sig_f=df['f'].sum()

  print("calculating 'tf','idf' and 'tf-idf':")
  for word in tqdm(vocab):
      # the index of the word
      index = df.index[df['word'] == word][0]
      f = df.at[index, 'f']
      nt = df.at[index, 'nt']
      # Calculate the tf value
      tf = f / sig_f
      # shape[0] is the numebr of rows, aka the numebr of tweets
      n = dense_matrix.shape[0]
      # Calculate the idf value
      idf = np.log(n / (nt + 1))
      # Calculate the tf-idf value
      tf_idf = tf * idf
      # update the word's ramaining information to the dataframe
      df.at[index, 'tf'] = tf
      df.at[index, 'idf'] = idf
      df.at[index, 'tf-idf'] = tf_idf
  print("done!")

  #sort in descending order
  df = df.sort_values('tf-idf', ascending=False)
  #add vocabulary
  columns, meanings = get_terms()
  for i in range(len(columns)):
    df.at[df.index[i], "Column"] = columns[i]
    df.at[df.index[i], "Meaning"] = meanings[i]
  # write to csv
  df.to_csv(f'/content/drive/MyDrive/model_files/tf idf charts/{preprocessing}_words_tfidf_chart.csv', encoding='utf-8-sig',index=False)
  # write to h5
  df.to_hdf(f'/content/drive/MyDrive/model_files/tf idf charts/{preprocessing}_words_tfidf_chart.h5', key='words_df', mode='w')


In [ ]:
#itirate over each preprocessing method and writes the tfidf chart
for preprocessing, vectorizer in VECTORIZERS.items():
  print("------------------------------------------")
  print(f"writing the {preprocessing} tf-idf chart:")
  write_tf_idf_chart(preprocessing, vectorizer)
  print("------------------------------------------")

------------------------------------------
writing the N tf-idf chart:
Getting words from tweets:


100%|██████████| 12825/12825 [00:00<00:00, 73209.73it/s]


done!
calculating 'nt' and 'f':


100%|██████████| 19984/19984 [20:40<00:00, 16.11it/s]


done!
calculating 'tf','idf' and 'tf-idf':


...

done!
calculating 'nt' and 'f':


100%|██████████| 19771/19771 [20:23<00:00, 16.16it/s]


done!
calculating 'tf','idf' and 'tf-idf':


100%|██████████| 19771/19771 [00:46<00:00, 427.22it/s]


done!
------------------------------------------


In [ ]:
# writes the abbreviations file
with open("/content/drive/MyDrive/model_files/data/abbreviations_not_expanded_.txt", "w") as f:
    train_tweets = read_train_tweets()
    words=get_words(train_tweets["Tweet"],"special characters removal")
    abbreviations=[]
    print(words)
    for word in words:
        # Check if the word matches the regex and includes the character "?
        if (("\"" in word)or ("\'" in word) or ("״" in word)) and not( word in abbreviations ):
            # Write the word to the text file
            abbreviations.append(word)
            f.write(word + "\n")

100%|██████████| 12825/12825 [00:00<00:00, 79120.35it/s]
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



###Vectorize

In [ ]:
NUMBER_OF_WORDS = [5000,4000,3000,2000,1000]  # the number of the highest tfidf words
CLASSIFY = [
"קבוצת גיל"
,"מין"
,"0-ימין"
,"1-ימין מרכז"
,"2-שמאל מרכז"
,"3-שמאל"
,"0-ליכוד"
,"1-יש עתיד"
,"2-הציונות הדתית"
,"3-מרצ"
,"4-תקווה חדשה"
,"5-ימינה"
,"6- ישראל ביתנו"
,"7-כחול לבן"
,"8-ש\"ס"
,"9-הרשימה המשותפת"
,"10-העבודה"
,"11-רע\"מ"
,"12-ח\"כ בודד"
,"0-ספרדי"
,"1-אשכנזי"
,"דתי אורתודוקסי"
,"לא חרדי-0"
,"חרדי-1"
,"יהודי"]  # the classification attribute of the tweet vectors
SPECIAL_CHARCATERS_REMOVAL=r"(?u)\b\w+[\"\'״]*\w\b" # tokenize by " " and "/n" and allow abbreviations
TOKEN_PATTERN="[^ \n]+" # baseline tokenize, only by space and end of row

# Gets all the stop words from the file
with open("/content/drive/MyDrive/model_files/data/stop_words.txt", 'r') as f:
        STOP_WORDS = f.read().splitlines()

# Insert all the abbreviations into a dictionary
ABBREVIATIONS = {}
with open("/content/drive/MyDrive/model_files/data/abbreviations.txt") as f:
    for line in f:
        key, value = line.split("-")
        ABBREVIATIONS[key.strip()] = value.strip()

# To preprocess open abbrevaitons
def expand_abbreviations(text):
    """
    a function that gets a text and replace all abbreviations with thier expnaded form
    """
    for match in re.finditer(SPECIAL_CHARCATERS_REMOVAL, text):
        abbreviation = match.group()
        normelized_abbreviation=abbreviation
        # normalize all " 
        if "״" in abbreviation:
          normelized_abbreviation=abbreviation.replace("״","\"")
        if "''" in abbreviation:
          normelized_abbreviation=abbreviation.replace("''","\"")
        # if its in the, replace with expanded form
        if normelized_abbreviation in ABBREVIATIONS:
            text = text.replace(abbreviation, ABBREVIATIONS[normelized_abbreviation])
    return text

In [ ]:
def read_train_tweets():
  """
  reads the data from the train tweets sheet
  :return: the data as pandas data set
  """
  store = pd.HDFStore('/content/drive/MyDrive/model_files/data/train_tweets_set.h5')
  df = store.select('train_tweets_set')
  store.close()

  columns_to_include = ["User","Tweet"]
  df = df[columns_to_include]
  df.dropna(subset=["Tweet"], inplace=True)
  return df

def read_test_tweets():
  """
  reads the data from the test tweets sheet
  :return: the data as pandas data set
  """
  store = pd.HDFStore('/content/drive/MyDrive/model_files/data/test_tweets_set.h5')
  df = store.select('test_tweets_set')
  store.close()

  columns_to_include = ["User","Tweet"]
  df = df[columns_to_include]
  df.dropna(subset=["Tweet"], inplace=True)
  return df

def read_train_knesset_members():
  """
  reads the data from the train knesst members sheet
  :return: the data as pandas data set
  """
  train_knesset_members= pd.read_hdf('/content/drive/MyDrive/model_files/data/train_knesset_members_set.h5',key='train_knesset_members_set',)
  return train_knesset_members

def read_test_knesset_members():
  """
  reads the data from the test knesst members sheet
  :return: the data as pandas data set
  """
  test_knesset_members= pd.read_hdf('/content/drive/MyDrive/model_files/data/test_knesset_members_set_set.h5',key='test_knesset_members_set',)
  return test_knesset_members

def write_classification(tweets, knesset, classification,set):
  """
  return the tweet vector with the classification
  :param tweets: the tweets set
  :param knesset_members_train_set: the knesset members information
  :param classification: the desired classification
  :return: the classified tweets dataframe
  """
  classified_tweets_df = pd.DataFrame(columns=["class: "+classification])
  print(f"writing {set} tweets classification:")
  for index in tqdm(range(len(tweets.index))):
      username = tweets.iat[index,0]
      tweet = tweets.iat[index, 1]
      if not pd.isna(tweet) and not pd.isna(username):
          classification_index = knesset.index[knesset['שם משתמש'] == username][0]
          tweet_classification = knesset.at[classification_index, classification]
          tweet_classification = str(tweet_classification)
          if cls=="קבוצת גיל":
            if tweet_classification == "עד 50":
              tweet_classification = 0
            else:
              tweet_classification = 1
          else:
              tweet_classification = re.findall(r'\d+',tweet_classification)[0]
          classified_tweets_df.loc[len(classified_tweets_df.index)] = [tweet_classification]
  print("done!")
  classified_tweets_df.to_csv(f'/content/drive/MyDrive/model_files/classification/{set}_tweets_classification_{classification}.csv', encoding='utf-8-sig', index=False)
  classified_tweets_df.to_hdf(f'/content/drive/MyDrive/model_files/classification/{set}_tweets_classification_{classification}.h5', key='classification', mode='w')

def write_vectors(tfidf_vocabulary,vectors,set,amount,preprocessing):
  """
  write the given set tfidf vectors by the tf idf vocabulary
  """
  dense_vectors = vectors.toarray()
  df = pd.DataFrame(dense_vectors, columns=tfidf_vocabulary)
  df.to_csv(f'/content/drive/MyDrive/model_files/vectors/{preprocessing}_{set}_tweets_vectors_{amount}.csv', encoding='utf-8-sig',index=False)
  df.to_hdf(f'/content/drive/MyDrive/model_files/vectors/{preprocessing}_{set}_tweets_vectors_{amount}.h5', key='vectors', mode='w')

def get_words(amount,preprocessing):
  """
  get the highest tf-idf words by a given amount from the df-idf chart
  :param amount: the amount of words
  :return: the given amount of words with the highest TF-IDF value
  """
  words = pd.read_hdf(f'/content/drive/MyDrive/model_files/tf idf charts/{preprocessing}_words_tfidf_chart.h5',key="words_df")
  words = words.head(amount)
  words = words['word'].tolist()
  return words

In [ ]:
train_tweets = read_train_tweets()
train_knesset_members=read_train_knesset_members()
test_tweets = read_test_tweets()
test_knesset_members=read_test_knesset_members()
for amount in NUMBER_OF_WORDS:
  VECTORIZERS={ # the tf-idf vectorizers by preprocessing method
    'N' : TfidfVectorizer(vocabulary=get_words(amount,'N'),lowercase=False, token_pattern=TOKEN_PATTERN),#None, baseline
    # only one preprocessing method
    'S':TfidfVectorizer(vocabulary=get_words(amount,'S'),lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS),#Stop word removal
    'C':TfidfVectorizer(vocabulary=get_words(amount,'C'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL),#special Characters removal
    'O':TfidfVectorizer(vocabulary=get_words(amount,'O'),lowercase=False, token_pattern=TOKEN_PATTERN, preprocessor=expand_abbreviations),#Open abbreviations
    # pairing preprocssing methods
    #'SC':TfidfVectorizer(vocabulary=get_words(amount,'SC'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS),
    #'SO':TfidfVectorizer(vocabulary=get_words(amount,'SO'),lowercase=False, token_pattern=TOKEN_PATTERN,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
    #'CO':TfidfVectorizer(vocabulary=get_words(amount,'CO'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,preprocessor=expand_abbreviations),
    # all preprocessing methods
    #'SCO':TfidfVectorizer(vocabulary=get_words(amount,'SCO'),lowercase=False, token_pattern=SPECIAL_CHARCATERS_REMOVAL,stop_words=STOP_WORDS,preprocessor=expand_abbreviations),
  }
  for preprocessing, vectorizer in tqdm(VECTORIZERS.items()):
    tfidf_vocabulary=get_words(amount,preprocessing)
    train_vectors = vectorizer.fit_transform(train_tweets["Tweet"])
    write_vectors(tfidf_vocabulary,train_vectors,"train",amount,preprocessing)

    test_vectors = vectorizer.fit_transform(test_tweets["Tweet"])
    write_vectors(tfidf_vocabulary,test_vectors,"test",amount,preprocessing)

 25%|██▌       | 1/4 [01:33<04:39, 93.07s/it]/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['יד', 'ידי'] not in stop_words.
  warnings.warn(
 75%|███████▌  | 3/4 [04:54<01:39, 99.15s/it]/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:1322: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(
100%|██████████| 4/4 [02:38<00:00, 39.52s/it]


In [ ]:
train_tweets = read_train_tweets()
train_knesset_members=read_train_knesset_members()
test_tweets = read_test_tweets()
test_knesset_members=read_test_knesset_members()
for cls in CLASSIFY:
  print("writing "+cls)
  write_classification(train_tweets,train_knesset_members,cls, "train")
  write_classification(test_tweets,test_knesset_members,cls, "test")

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


writing קבוצת גיל
writing train tweets classification:


100%|██████████| 12825/12825 [00:26<00:00, 490.69it/s]


done!


/usr/local/lib/python3.8/dist-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_values] [items->Index(['class: קבוצת גיל'], dtype='object')]

  pytables.to_hdf(


...

done!
writing יהודי
writing train tweets classification:


100%|██████████| 12825/12825 [00:26<00:00, 482.81it/s]


done!
writing test tweets classification:


100%|██████████| 4330/4330 [00:08<00:00, 510.92it/s]


done!


##ML Methods

###Results

In [ ]:
RANDOM_STATE = 1 # in order to get consistent results for the random forest algorithem
CLASSIFY = [
#"קבוצת גיל"//all
#,"מין"//all
#,"0-ימין",//all
#"1-ימין מרכז"
#,"2-שמאל מרכז"//all
#,"3-שמאל"
#,"0-ליכוד"//all
#,"1-יש עתיד"
#,"2-הציונות הדתית"//all
#,"3-מרצ"//all
#,"4-תקווה חדשה"//all
#,"5-ימינה"//all
#,"6- ישראל ביתנו"//all
#,"7-כחול לבן"//all
#,"8-ש\"ס"//all
#,"9-הרשימה המשותפת"//all
#,"10-העבודה"//all
#,"11-רע\"מ"
#,
"12-ח\"כ בודד"
#,"0-ספרדי"//all
#,"1-אשכנזי"//all
#,"דתי אורתודוקסי"//all
#,"לא חרדי-0"//all
#,"חרדי-1"//all
#,"יהודי"//all
]  # the classification attribute of the tweet vectors
NUM_OF_WORDS=[1000,2000,3000,4000,5000]
ML_CLASSIFIERS= [ # the classifers we use
"RF"
#,"SVM"
,"MLP"
,"MNB"
,"LR"
]
OVERSAMPLERS=[#"none",
              "random",
              #"smote",
              #"adasyn"
              ]
# the preprocessing methods
PREPROCESSORS=[#'N','S','C',
               'O'
               ]#,'SC','SO','CO','SCO']
RANDOM_STATE=42

In [ ]:
def get_sets_vectors(class_name,amount,preprocessing,oversampler):
  """
  reads and returns the vectors and classifcation of the train and test sets.
  """
  x_train=pd.read_hdf(f'/content/drive/MyDrive/model_files/vectors/{preprocessing}_train_tweets_vectors_{amount}.h5',key='vectors')
  y_train=pd.read_hdf(f'/content/drive/MyDrive/model_files/classification/train_tweets_classification_{class_name}.h5',key='classification').iloc[:,0] 
  y_train=np.array(y_train).astype(int)                          
  x_test=pd.read_hdf(f'/content/drive/MyDrive/model_files/vectors/{preprocessing}_test_tweets_vectors_{amount}.h5',key='vectors')
  y_test=pd.read_hdf(f'/content/drive/MyDrive/model_files/classification/test_tweets_classification_{class_name}.h5',key='classification').iloc[:,0]
  y_test=np.array(y_test).astype(int)

  if oversampler != "none":
    if oversampler == "random":
      oversampler = RandomOverSampler(random_state=RANDOM_STATE)
    if oversampler == "smote":
      oversampler = SMOTE(random_state=RANDOM_STATE)
    if oversampler == "adasyn":
      oversampler = ADASYN(random_state=RANDOM_STATE)

    x_train, y_train = oversampler.fit_resample(x_train, y_train)      
  return x_train, x_test, y_train, y_test


def score_classifier(y_test, predictions, classifier,f1_classifiers_scores):
  """
  returns the given classifers score by comparing the predictions and true classifies.
  """
  # calculate the classifier's score
  f1 = f1_score(y_test, predictions)
  #precision = precision_score(y_test, predictions, zero_division=0)
  #accuracy = accuracy_score(y_test, predictions)
  #recall = recall_score(y_test, predictions)
  #roc_auc = roc_auc_score(y_test, predictions)

  # add the classifer score to the dictionary
  f1_classifiers_scores[classifier] = f1
  #precision_classifiers_scores[classifier] = precision
  #accuracy_classifiers_scores[classifier] = accuracy
  #recall_classifiers_scores[classifier] = recall
  #auc_classifiers_scores[classifier] = roc_auc

  #return f1, precision, accuracy, recall, roc_auc
  return f1,0,0,0,0

def print_classifier_score(f1, precision, accuracy, recall, roc_auc, classifier):
  """
  prints the scores of the given classifier.
  """
  print("---------------------")
  print(classifier + " score:")
  print("F1: " + str(f1))
  print("Precision: " + str(precision))
  print("Accuracy: " + str(accuracy))
  print("Recall: " + str(recall))
  print("ROC Under Curve: " + str(roc_auc))
  print("---------------------")

def run_classifier(x_train, x_test, y_train, y_test,classifier,f1_classifiers_scores):
  """
  runs the specified classifier with the given train and test sets.
  """
  # create a scikit classifier by the given classifier name
  if classifier=="RF":
      clf = RandomForestClassifier(random_state=RANDOM_STATE)
  elif classifier=="SVM":
      clf = SVC()
  elif classifier=="MLP":
      clf = MLPClassifier()
  elif classifier=="MNB":
      clf = MultinomialNB()
  elif classifier=="LR":
      clf = LogisticRegression()
  else:
      raise ValueError("unknown classifier given")
  clf.fit(x_train, y_train)
  predictions = clf.predict(x_test)

  f1, precision, accuracy, recall, roc_auc = score_classifier(y_test, predictions, classifier,f1_classifiers_scores)
  #print_classifier_score(f1, precision, accuracy, recall, roc_auc, classifier)


def ml_classifiers(x_train, x_test, y_train, y_test,classifiers):
  """
  creates the classifers and prints their scores. 
  """
  # scores dictionaries to store and summary by measure
  f1_classifiers_scores = {}
  #accuracy_classifiers_scores = {}
  for classifier in tqdm(classifiers):
    run_classifier(x_train, x_test, y_train, y_test,classifier,f1_classifiers_scores)

  return f1_classifiers_scores

def create_score_chart(cls,oversampling,preprocessing):
  df_columns = []
  # Create a data frame with the title row
  df_columns.append("Number of words")
  df_columns = df_columns + ML_CLASSIFIERS
  df = pd.DataFrame(columns=df_columns)
    
  # Create an array of empty strings
  empty_strings = np.empty(len(df_columns), dtype='U64')
  empty_strings[:] = ''

  # Set the first element of the array to a specific value
  empty_strings[0] = "f1 results of "+oversampling+ " oversampling after preprocess of " + cls + " by " + preprocessing
  title = pd.DataFrame(data=[empty_strings], columns=df.columns)
  for amount in NUM_OF_WORDS:
      print(f"running ml methods for {amount} of words classifying {cls} with preprocessing '{preprocessing}'...")
      x_train, x_test, y_train, y_test=get_sets_vectors(cls,amount,preprocessing,oversampling)
      f1_classifiers_scores = ml_classifiers(x_train, x_test, y_train, y_test,ML_CLASSIFIERS)
      df_row = []
      df_row.append(amount)
      df_row = df_row + list(f1_classifiers_scores.values())
      df.loc[len(df.index)] = df_row
  print(tabulate(df, headers='keys', tablefmt='psql'))
  # Concatenate the two data frames
  print(cls+" "+oversampling+" "+preprocessing+":")
  header = pd.DataFrame(data=[df_columns], columns=df_columns)
  csv_df = pd.concat([title,header,df], ignore_index=True)
  csv_df.to_csv(f'/content/drive/MyDrive/model_files/results/{oversampling}/csv/{cls}_by_{preprocessing}_f1_results.csv', header=False, encoding='utf-8-sig', index=False)
  df.to_hdf(f'/content/drive/MyDrive/model_files/results/{oversampling}/h5/{cls}_by_{preprocessing}_f1_results.h5', key='results', mode='w')
  print("finished wreiting "+cls+" "+oversampling+" "+preprocessing+":")

In [ ]:
tf.test.gpu_device_name()
for cls in CLASSIFY:
 for oversampling in OVERSAMPLERS:
  processes = [] #open a process for each preprocessing method
  for preprocessing in PREPROCESSORS:
    p = Process(target=create_score_chart, args=(cls,oversampling,preprocessing))
    p.start()
    processes.append(p)

    # wait for all processes to finish
  for p in processes:
    p.join()
  print("finished oversampling "+oversampling)

running ml methods for 1000 of words classifying 12-ח"כ בודד with preprocessing 'O'...


100%|██████████| 4/4 [01:08<00:00, 17.17s/it]


running ml methods for 2000 of words classifying 12-ח"כ בודד with preprocessing 'O'...


100%|██████████| 4/4 [01:50<00:00, 27.57s/it]


running ml methods for 3000 of words classifying 12-ח"כ בודד with preprocessing 'O'...


...

running ml methods for 4000 of words classifying 12-ח"כ בודד with preprocessing 'O'...


100%|██████████| 4/4 [02:45<00:00, 41.37s/it]


running ml methods for 5000 of words classifying 12-ח"כ בודד with preprocessing 'O'...


100%|██████████| 4/4 [03:16<00:00, 49.02s/it]


+----+-------------------+------+-------+-------+------+
|    |   Number of words |   RF |   MLP |   MNB |   LR |
|----+-------------------+------+-------+-------+------|
|  0 |              1000 |    0 |     0 |     0 |    0 |
|  1 |              2000 |    0 |     0 |     0 |    0 |
|  2 |              3000 |    0 |     0 |     0 |    0 |
|  3 |              4000 |    0 |     0 |     0 |    0 |
|  4 |              5000 |    0 |     0 |     0 |    0 |
+----+-------------------+------+-------+-------+------+
12-ח"כ בודד random O:
finished wreiting 12-ח"כ בודד random O:
finished oversampling random


###One hot encoded summary

In [ ]:
ONE_HOT_ENCODED={#all classification that are one hot encoded should be macro averged
    "דעה":
          ["0-ימין"
          ,"1-ימין מרכז"
          ,"2-שמאל מרכז"
          ,"3-שמאל"],
    "מפלגה":
          ["0-ליכוד"
          ,"1-יש עתיד"
          ,"2-הציונות הדתית"
          ,"3-מרצ"
          ,"4-תקווה חדשה"
          ,"5-ימינה"
          ,"6- ישראל ביתנו"
          ,"7-כחול לבן"
          ,"8-ש\"ס"
          ,"9-הרשימה המשותפת"
          ,"10-העבודה"
          ,"11-רע\"מ"
          ,"12-ח\"כ בודד"],
    "עדה":
          ["0-ספרדי"
          ,"1-אשכנזי"],
    "חרדי":
          ["לא חרדי-0"
          ,"חרדי-1"]
    }
PREPROCESSORS=['N','S','C','O']#,'SC','SO','CO','SCO']
ML_CLASSIFIERS= [ # the classifers we use
"RF"
#,"SVM"
,"MLP"
,"MNB"
,"LR"
]
NUM_OF_WORDS=[1000,2000,3000,4000,5000]
score="f1"
OVERSAMPLERS=[#"none",
              "random",
              #"smote",
              #"adasyn"
              ]

In [ ]:
for oversampling in OVERSAMPLERS:
  print(oversampling)
  for cls,attributes in ONE_HOT_ENCODED.items():
    print(cls)
    for preprocessing in PREPROCESSORS:
      print(preprocessing)
      df_columns = []
      # Create a data frame with the title row
      df_columns.append("Number of words")
      df_columns = df_columns + ML_CLASSIFIERS
      df = pd.DataFrame(columns=df_columns)
      # Create an array of empty strings
      empty_strings = np.empty(len(df_columns), dtype='U64')
      empty_strings[:] = ''
      # Set the first element of the array to a specific value
      empty_strings[0] = "f1 results of " + cls + " by " + preprocessing
      title = pd.DataFrame(data=[empty_strings], columns=df.columns)
      for num in NUM_OF_WORDS:
        row={}
        row["Number of words"] = num
        for classifier in ML_CLASSIFIERS:
          row[classifier] = 0;
          for attr in attributes:
            ml_results = pd.read_hdf(f'/content/drive/MyDrive/model_files/results/{oversampling}/h5/{attr}_by_{preprocessing}_{score}_results.h5', key='results')
            index = ml_results.loc[ml_results["Number of words"] == num].index[0]
            f1_score=ml_results.at[index,classifier]
            row[classifier] = row[classifier] +f1_score
          row[classifier] = row[classifier] / len(attributes)
        df.loc[len(df.index)] = row
      print(tabulate(df, headers='keys', tablefmt='psql'))
      # Concatenate the two data frames
      header = pd.DataFrame(data=[df_columns], columns=df_columns)
      csv_df = pd.concat([title,header,df], ignore_index=True)
      csv_df.to_csv(f'/content/drive/MyDrive/model_files/results/{oversampling}/csv/{cls}_by_{preprocessing}_f1_results.csv', header=False, encoding='utf-8-sig', index=False)
      df.to_hdf(f'/content/drive/MyDrive/model_files/results/{oversampling}/h5/{cls}_by_{preprocessing}_f1_results.h5', key='results', mode='w') 
            
            


random
מפלגה
O
+----+-------------------+----------+----------+----------+----------+
|    |   Number of words |       RF |      MLP |      MNB |       LR |
|----+-------------------+----------+----------+----------+----------|
|  0 |              1000 | 0.108213 | 0.153857 | 0.19109  | 0.185234 |
|  1 |              2000 | 0.117416 | 0.162423 | 0.201516 | 0.192066 |
|  2 |              3000 | 0.119185 | 0.152075 | 0.204329 | 0.194944 |
|  3 |              4000 | 0.114563 | 0.155445 | 0.205602 | 0.197283 |
|  4 |              5000 | 0.116366 | 0.156604 | 0.20811  | 0.202834 |
+----+-------------------+----------+----------+----------+----------+


/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_values] [items->Index(['Number of words'], dtype='object')]

  pytables.to_hdf(


##Summary

In [ ]:
CLASSIFY = [
"קבוצת גיל"
,"מין"
,"דעה"
,"מפלגה"
,"עדה"
,"דתי אורתודוקסי"
,"חרדי"
,"יהודי"
]  # the classification attribute of the tweet vectors
PREPROCESSORS=['N','S','C','O']#,'SC','SO','CO','SCO']
ML_CLASSIFIERS= [ # the classifers we use
"RF"
#,"SVM"
,"MLP"
,"MNB"
,"LR"
]
NUM_OF_WORDS=[1000,2000,3000,4000,5000]
score="f1"
OVERSAMPLERS=["none",
              "random",
              "smote",
              "adasyn"
              ]

###Best of each chart

In [ ]:
df_columns = ["Preprocessing","Classifier","Number of words","Score"]
for oversampling in OVERSAMPLERS:
  print(f"starting oversampling {oversampling}")
  for cls in CLASSIFY:
    print(f"{cls}:")
    summary = pd.DataFrame(columns=df_columns)
    for preprocessing in PREPROCESSORS:
        ml_results = pd.read_hdf(f'/content/drive/MyDrive/model_files/results/{oversampling}/h5/{cls}_by_{preprocessing}_{score}_results.h5', key='results')
        max_value_index = ml_results[ml_results.columns[1:]].stack().idxmax()
        row_index= int(max_value_index[0])
        column_index= ml_results.columns.get_loc(max_value_index[1])
        row={"Preprocessing": preprocessing,"Classifier": max_value_index[1],"Number of words": ml_results.iat[row_index,0],"Score":ml_results.iat[row_index,column_index] }
        summary.loc[len(summary.index)] = row
    summary = summary.sort_values('Score', ascending=False)
    print(tabulate(summary, headers='keys', tablefmt='psql',showindex=False))
    summary.to_csv(f'/content/drive/MyDrive/model_files/summary/charts/{oversampling}/csv/{cls}_{score}_results_summary.csv', encoding='utf-8-sig', index=False)
    summary.to_hdf(f'/content/drive/MyDrive/model_files/summary/charts/{oversampling}/h5/{cls}_{score}_results_summary.h5', key='summary', mode='w') 


starting oversampling none
קבוצת גיל:
+-----------------+--------------+-------------------+----------+
| Preprocessing   | Classifier   |   Number of words |    Score |
|-----------------+--------------+-------------------+----------|
| N               | MNB          |              1000 | 0.738924 |
| C               | MNB          |              1000 | 0.736194 |
| S               | MNB          |              1000 | 0.732587 |
| O               | MNB          |              1000 | 0.732443 |
+-----------------+--------------+-------------------+----------+
מין:
+-----------------+--------------+-------------------+----------+
| Preprocessing   | Classifier   |   Number of words |    Score |
|-----------------+--------------+-------------------+----------|
| N               | MLP          |              1000 | 0.417332 |
| O               | MLP          |              1000 | 0.405753 |
| C               | MLP          |              1000 | 0.404232 |
| S               | MLP          

/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['Preprocessing', 'Classifier', 'Number of words'], dtype='object')]

  pytables.to_hdf(


מפלגה:
+-----------------+--------------+-------------------+----------+
| Preprocessing   | Classifier   |   Number of words |    Score |
|-----------------+--------------+-------------------+----------|
| C               | MLP          |              3000 | 0.159004 |
| O               | MLP          |              5000 | 0.149158 |
| N               | MLP          |              5000 | 0.14132  |
| S               | MLP          |              4000 | 0.141224 |
+-----------------+--------------+-------------------+----------+
עדה:
+-----------------+--------------+-------------------+----------+
| Preprocessing   | Classifier   |   Number of words |    Score |
|-----------------+--------------+-------------------+----------|
| O               | MLP          |              5000 | 0.482343 |
| C               | MLP          |              2000 | 0.481961 |
| N               | MLP          |              1000 | 0.480899 |
| S               | MLP          |              4000 | 0.475917 

###Best of each class

In [ ]:
df_columns = ["Oversampling","Preprocessing","Classifier","Number of words","Score"]
for cls in CLASSIFY:
    print(f"{cls}:")
    class_summary = pd.DataFrame(columns=df_columns)
    for oversampling in OVERSAMPLERS:
      summary= pd.read_hdf(f'/content/drive/MyDrive/model_files/summary/charts/{oversampling}/h5/{cls}_{score}_results_summary.h5', key='summary')
      summary["Oversampling"] = oversampling
      class_summary = pd.concat([class_summary,summary], ignore_index=True)
    class_summary = class_summary.sort_values('Score', ascending=False)
    class_summary=class_summary.head(4)
    print(tabulate(class_summary, headers='keys', tablefmt='psql',showindex=False))
    class_summary.to_csv(f'/content/drive/MyDrive/model_files/summary/classes/csv/{cls}_{score}_results_summary.csv', encoding='utf-8-sig', index=False)
    class_summary.to_hdf(f'/content/drive/MyDrive/model_files/summary/classes/h5/{cls}_{score}_results_summary.h5', key='class_summary', mode='w')      


 



קבוצת גיל:
+----------------+-----------------+--------------+-------------------+----------+
| Oversampling   | Preprocessing   | Classifier   |   Number of words |    Score |
|----------------+-----------------+--------------+-------------------+----------|
| none           | N               | MNB          |              1000 | 0.738924 |
| none           | C               | MNB          |              1000 | 0.736194 |
| adasyn         | C               | MNB          |              1000 | 0.733837 |
| none           | S               | MNB          |              1000 | 0.732587 |
+----------------+-----------------+--------------+-------------------+----------+
מין:
+----------------+-----------------+--------------+-------------------+----------+
| Oversampling   | Preprocessing   | Classifier   |   Number of words |    Score |
|----------------+-----------------+--------------+-------------------+----------|
| smote          | N               | LR           |              1000 |

/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py:2703: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['Oversampling', 'Preprocessing', 'Classifier', 'Number of words'], dtype='object')]

  pytables.to_hdf(


מפלגה:
+----------------+-----------------+--------------+-------------------+----------+
| Oversampling   | Preprocessing   | Classifier   |   Number of words |    Score |
|----------------+-----------------+--------------+-------------------+----------|
| smote          | C               | MNB          |              5000 | 0.220464 |
| random         | C               | MNB          |              4000 | 0.219324 |
| adasyn         | C               | MNB          |              4000 | 0.219125 |
| adasyn         | O               | MNB          |              5000 | 0.210642 |
+----------------+-----------------+--------------+-------------------+----------+
עדה:
+----------------+-----------------+--------------+-------------------+----------+
| Oversampling   | Preprocessing   | Classifier   |   Number of words |    Score |
|----------------+-----------------+--------------+-------------------+----------|
| adasyn         | N               | LR           |              4000 | 0.5

###Best Configurations

In [ ]:
oversamplers_dict={}
preprocesses_dict={}
classifiers_dict={}
num_of_words_dict={}
for cls in CLASSIFY:
    class_summary= pd.read_hdf(f'/content/drive/MyDrive/model_files/summary/classes/h5/{cls}_{score}_results_summary.h5', key='class_summary')
    class_summary = class_summary.reset_index()
    for i in range(len(class_summary.index)):
      factor = len(class_summary.index) - i
      oversmpling = class_summary.at[i,"Oversampling"]
      preprocess = class_summary.at[i,"Preprocessing"]
      classifier = class_summary.at[i,"Classifier"]
      num_of_words = class_summary.at[i,"Number of words"]

      if oversmpling in oversamplers_dict:
        oversamplers_dict[oversmpling] = oversamplers_dict[oversmpling] + factor
      else:
        oversamplers_dict[oversmpling] = factor
      
      if preprocess in preprocesses_dict:
        preprocesses_dict[preprocess] = preprocesses_dict[preprocess] + factor
      else:
        preprocesses_dict[preprocess] = factor
      
      if classifier in classifiers_dict:
        classifiers_dict[classifier] = classifiers_dict[classifier] + factor
      else:
        classifiers_dict[classifier] = factor
      
      if num_of_words in num_of_words_dict:
        num_of_words_dict[num_of_words] = num_of_words_dict[num_of_words] + factor
      else:
        num_of_words_dict[num_of_words] = factor

oversamplers_dict = dict(sorted(oversamplers_dict.items(), key=lambda x: x[1],reverse=True))
preprocesses_dict = dict(sorted(preprocesses_dict.items(), key=lambda x: x[1],reverse=True))
classifiers_dict = dict(sorted(classifiers_dict.items(), key=lambda x: x[1],reverse=True))
num_of_words_dict = dict(sorted(num_of_words_dict.items(), key=lambda x: x[1],reverse=True))


print(oversamplers_dict)
print(preprocesses_dict)
print(classifiers_dict)
print(num_of_words_dict)

{'adasyn': 33, 'random': 22, 'smote': 14, 'none': 11}
{'C': 39, 'N': 19, 'S': 15, 'O': 7}
{'MNB': 41, 'LR': 22, 'RF': 17}
{1000.0: 34, 5000: 17, 4000: 11, 2000.0: 9, 3000.0: 9}
